## M343 - Applied Number Theory
## Assignment #2
### Abigail Johnson ( amj2694 )
### Fall 2016

In [13]:
from __future__ import division
from math import factorial
import sys
import os
import math
import numpy as np
import re

IN_FILE = 'input.txt'
OUT_FILE = 'output.txt'


# opens file specified by name, returns contents in array of strings
def process_file( file_name ):
    return open(file_name).read().splitlines()


# Returns gcd(x,y) using Euclidean algorithm
# Uses fact that (1) m = nq + r where 0 <= r < n ( for m,n,q,r in Z )
#                (2) gcd(m,n) = gcd(n,r)    
def gcd(x,y):
    # slight performance optimization ( x > 0 added to account neg num)
    if y > x and x > 0 :
        return gcd(y,x)
    elif x % y == 0:
        return abs(y)
    else:
        return abs(gcd( y , x % y ))
    
    
# Computes addition or multiplication (denoted by op) for x and y in ring m    
def addition_multiplication_ring(m, op, x, y):
    if op == "*":
        result = ( x * y) % m
    else:
        result = ( x + y) % m
    return result


# Decode Message Encrypted with Shift Cypher
def decode_shift_cipher(message):
    # 1. Preprocess message and setup dicts 
    message = message.lower()
    message = re.sub(r'[^a-zA-Z ]+', '', message)
    
    # 2. Set up dict objects used for processing 
    alph_dict = { 'a' : 0, 'b': 1, 'c': 2, 'd': 3, 'e' : 4, 'f' : 5, 'g' : 6, 'h' : 7, 'i' : 8, 
                 'j' : 9, 'k' : 10, 'l' : 11, 'm' : 12, 'n' : 13, 'o' : 14, 'p' : 15, 'q' : 16, 
                 'r' : 17, 's' : 18 , 't' : 19, 'u' : 20, 'v' : 21, 'w' : 22, 'x' : 23, 'y' : 24, 'z' : 25 }
    alph_dict_inv = { v: k for k, v in alph_dict.items() } # invert to allow reverse indexing 
    english_dict = enchant.Dict("en_US")
    shift_dict = {'key' : 0, 'valid_words' : 0, 'message': '' }
    
    # 2. Shift messasge 0-25 times, determine and record num valid words seen in shift_dict
    for key in range(0,26):            
        message_shift = [ char if char == ' ' else alph_dict_inv[ ( (alph_dict[char] + key) % 26 ) ] for char in message ]
        message_shift = ''.join(message_shift)
        valid_words = 0
        for word in message_shift.split(' '):
            if english_dict.check(word):
                valid_words += 1
        if valid_words >= shift_dict['valid_words']:
            shift_dict['key'] = key
            shift_dict['valid_words'] = valid_words
            shift_dict['message'] = message_shift
    return shift_dict


# computes g^x using fast exponentiation
def fast_exponentiation(g, x):
    if x == 0:
        return 1
    elif x % 2 == 1:
        return g * fast_exponentiation( g, x-1 )
    else:
        exp = fast_exponentiation( g, x/2)
        return exp * exp



#### Problem 1 - Elgamal Encryption
Please write a computer program that performs Elgamal encryption. The input is a file “input.txt” that has p on the first line, g on the second line, m on the third line, and g^a (the value sent by Alice) on the fourth line. Output the result to “output.txt”.

In [48]:
# Computes a inverse mod p 
# using Euclidean Algorithm and fact that 
def inverse_mod_p( a, p ):
    inv = u_v_gcd(a,p)['u']
   
    if inv < 0 :
        #print( "Negative inverse : ", inv, " Normalizing to postive inverse : " , inv % p, " p = ", p)
        inv = inv % p
    
    return inv

# TODO: current does not handle b = 0 . fix it
# given a and b, returns gcd and u,v inc Integer st au + bv = gcd(a,b)
def u_v_gcd(a, b):
    u = 1; gcd = a; x = 0; y = b
    while True:
        if y == 0:
            v = ( gcd - ( a * u ) ) // b
            ret = {}
            ret['u'] = u ; ret['v'] = v ; ret['gcd'] = gcd
            return ret
        q = gcd // y 
        r = gcd % y
        s = u - ( q * x )
        u = x ; x = s; gcd = y ; y = r
    
# Encrypts plaintext using elgamal encription
# returns ( c1, c2 )
# where c1 = ( g ^ k ) mod p
#       c2 = ( m * A ^ k ) mod p
#            where A = (g ^ a ) 
def elgamal_encryption(p, g, m, g_pow_a):
    y = np.random.randint( 1, p )
    print("y : ", y)
    c1 = fast_exponentiation( g_pow_a, y)
    c2 = c1 * inverse_mod_p( m, p)
    return ( c1, c2 )

def main():
    args = open('input.txt').read().splitlines()
    p = int(args[0]); g = int(args[1]) ; m = int(args[2]) ; g_pow_a = int(args[3]) 
    result = elgamal_encryption( p, g, m, g_pow_a )
    print( "Elgamal Encryption with p = ", p, ", g = ", g, ", m = ", m, ", g_pow_a = ", g_pow_a, " :\n \t e(m) = ", result)
    open('output.txt', 'w').write( str(result) )
    
if  __name__ =='__main__':
    main()

{'u': 1, 'gcd': 1, 'v': -2}
Inverse :  3
1
y :  3
Elgamal Encryption with p =  5 , g =  2 , m =  3 , g_pow_a =  4  :
 	 e(m) =  (64, 128)


#### Problem 2 - Babystep Giantstep Algorithm
Write a computer program that implements the Babystep-Giantstep algorithm for solving the discrete log problem. The input is a file “in- put.txt” that has p on the first line, g on the second, and h on the third. Output the result to “output.txt”.

In [63]:

# Input: A cyclic group G of order p, having a generator g and an element h.
# Output: A value x st g^x congr h modulo p
# Run-time : O( n^(1/2) )
def baby_step_giant_step(p, g, h):

    # 1. Compute limits
    m = math.ceil( math.sqrt(p) )
    
    # 2. Giant Step
    giant_step = {}
    for i in range ( 0, m ):
        giant_step[ fast_exponentiation( g , i) % p ] = i
    
    # 3. Baby Step / Intersection detection
    g_inv = inverse_mod_p( g, p )
    u = g_inv ** m 
    for i in range ( 0, m ):
        baby_step = ( ( u ** i ) * h ) % p 
        if baby_step in giant_step:
            x = m * i + giant_step[ baby_step ]
            return x

print(baby_step_giant_step(17,3,1))    
#print(baby_step_giant_step(31,3,6))

# Test file : p = 5, g = 2 , h = 1
def main():
    args = open('input.txt').read().splitlines()
    p = int(args[0]) ; g = int(args[1]) ; h = int(args[2])  
    result = baby_step_giant_step( p, g, h )
    print( "Result of Babystep Giantstep ( p = ", p, " g = ", g, " h = ", h, ") :\n ", result)
    open('output.txt', 'w').write( str(result) )
    
if  __name__ =='__main__':
    main()

0
Result of Babystep Giantstep ( p =  5  g =  2  h =  3 ) :
  3


#### Problem 1.36
Compute 2^[ (p-1)/2 ] mod p for every prime 3 <= p < 20 and make conjectures about values. Prove your conjecture right.

In [40]:
primes = [ 3, 5, 7, 11, 13, 17, 19 ]
for p in primes:
    exp = ( p - 1 ) /2
    val =  ( fast_exponentiation(2, exp) ) % p
    print("2^[ (p-1)/2 ] mod p for p = ", p, " : ", val, " \n")
    

2^[ (p-1)/2 ] mod p for p =  3  :  2  

2^[ (p-1)/2 ] mod p for p =  5  :  4  

2^[ (p-1)/2 ] mod p for p =  7  :  1  

2^[ (p-1)/2 ] mod p for p =  11  :  10  

2^[ (p-1)/2 ] mod p for p =  13  :  12  

2^[ (p-1)/2 ] mod p for p =  17  :  1  

2^[ (p-1)/2 ] mod p for p =  19  :  18  

